# Advanced Lane Finding Project 

__The goals / steps of this project are the following:__ 


1- Compute the camera calibration matrix and distortion coefﬁcients given a set of chessboard images.  
2- Apply a distortion correction to raw images.  
3- Use gradients and color transforms to create a thresholded binary image.  
4- Apply a perspective transform to rectify binary image ("birds-eye view").  
5- Detect lane pixels and ﬁt to ﬁnd the lane boundary.  
6- Determine the curvature of the lane and vehicle position with respect to center.  
7- Warp the detected lane boundaries back onto the original image.  
8- Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.  


In [ ]:
# importing some useful packages
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2

from ipywidgets import widgets
from ipywidgets import interact, interactive, fixed, interact_manual

%matplotlib inline

## 1. Calibration of the camera

In [ ]:
def _build_full_paths(dir_root):
    return [dir_root+'/'+fn for fn in os.listdir(dir_root)]

# The process to calibrate the camera is split into 2 phases:
# - first find the corners of several images with different angles of a chessboards,
# - second, calibrate the camera and 
class CameraCalibrator:
    def __init__(self, nx=9, ny=6):
        self.nx = nx
        self.ny = ny
        self.mtx = None
        self.dist = None
        
    def _find_corners(self, filenames, show_images=False):
        objpoints, imgpoints = [], [] # 3d points in real world space, # 2d points in image plane.
    
        # Prepare obj points
        objp = np.zeros((self.nx * self.ny, 3), np.float32)
        objp[:,:2] = np.mgrid[0:self.nx, 0:self.ny].T.reshape(-1,2)

        img_size = None
    
        for fn in filenames:
            #reading in an image
            image = mpimg.imread(fn)
            gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)

            ret, corners = cv2.findChessboardCorners(gray, (self.nx,self.ny), None)
        
            if img_size is None:
                img_size = (image.shape[1], image.shape[0])
        
            if ret == True:
                objpoints.append(objp)
                imgpoints.append(corners)
                
                if show_images:
                    image = cv2.drawChessboardCorners(image, (self.nx, self.ny), corners, ret)
                    plt.imshow(image)
                    plt.figure()
    
        return objpoints, imgpoints, img_size
    
    def _calibrate_camera(self, objpoints, imgpoints, img_size):
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)
        return mtx, dist
    
    def calibrate(self, filenames):
        objpoints, imgpoints, img_size = self._find_corners( filenames, show_images=True )
        self.mtx, self.dist = self._calibrate_camera(objpoints, imgpoints, img_size)
        return self.mtx, self.dist
       
        
ccal1 = CameraCalibrator()
ccal1.calibrate( _build_full_paths('camera_cal/') );


## 2.  Apply a distortion correction to raw images.

### a. Compute undistort image

In [ ]:
def _show_dist_undist_images_by_2(images, title0='Original Image', title1='Undistorted Image'):
    for img, dst in images:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
        ax1.imshow(img, cmap='gray')
        ax1.set_title(title0, fontsize=30)
        ax2.imshow(dst, cmap='gray')
        ax2.set_title(title1, fontsize=30)

class Undistorter:
    def __init__(self, camera_calibrator):
        self.camera_calibrator = camera_calibrator
        
    def undistort_image(self, img):
        mtx, dist = self.camera_calibrator.mtx, self.camera_calibrator.dist
        return cv2.undistort(img, mtx, dist, None, mtx)
        
    def read_undistort_images(self, filenames):
        undistorted_imgs = []
    
        for fn in filenames:
            img = mpimg.imread(fn)
        
            undst = self.undistort_image(img)
            undistorted_imgs.append( (img, undst, ) )
        
        return undistorted_imgs
        
undistorter = Undistorter(ccal1)


In [ ]:
_show_dist_undist_images_by_2( undistorter.read_undistort_images([ 'camera_cal/calibration1.jpg' ]) )

### b. Compute perspective matrix

In [ ]:
class PerspectiveWarper:
    def __init__(self, undistorter):
        self.undistorter = undistorter
        self.M = None
        
    def compute_perspective_transform(self, src, dst):
        self.M = cv2.getPerspectiveTransform(src, dst)
    
    def warp_perspective(self, img):
        if self.M is None:
            raise ValueError("!!! The matrix M must be computed by method compute_perspective_transform() before calling warp_perspective()")
        
        undist = self.undistorter.undistort_image(img)
        img_size = (undist.shape[1], undist.shape[0])
        return cv2.warpPerspective(undist, self.M, img_size)


#### Perspective warper applied to chessboards

In [ ]:
chess_persp_warper = PerspectiveWarper(undistorter)

def chessboard_perspective_warper(perspective_warper, img, offset = 100, nx=9, ny=6, draw_corners=True):
    undistorter = perspective_warper.undistorter
    
    undist = undistorter.undistort_image(img)
    # Convert undistorted image to grayscale
    gray = cv2.cvtColor(undist, cv2.COLOR_RGB2GRAY)
    # Search for corners in the grayscaled image
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
    
    if ret == True:
        if draw_corners:
            cv2.drawChessboardCorners(undist, (nx, ny), corners, ret)
            
        img_size = (gray.shape[1], gray.shape[0])
        
        src = np.float32([corners[0], corners[nx-1], corners[-1], corners[-nx]])
        dst = np.float32([[offset, offset], [img_size[0]-offset, offset], 
                            [img_size[0]-offset, img_size[1]-offset], [offset, img_size[1]-offset]])
    
        perspective_warper.compute_perspective_transform(src, dst)
        return perspective_warper.warp_perspective(undist)
    
    return None
    
def read_perspective_warper_chess_images(perspective_warper, filenames):
    result = []
    for fn in filenames:
        img = mpimg.imread(fn)
        warped = chessboard_perspective_warper(perspective_warper, img)
        
        if warped is None:
            continue
        result.append( (img, warped,) )
        
    return result

_show_dist_undist_images_by_2( read_perspective_warper_chess_images(chess_persp_warper, _build_full_paths('camera_cal/')), title1='Warped Image' )

#### Perspective warper applied to road images

In [ ]:
persp_warper = PerspectiveWarper(undistorter)

# Compute the perspective matrix for the road:
def compute_road_perspective_matrix(perspective_warper):
    # src = np.float32([[535, 460], [750, 460], [1480, 720], [-180, 720]])
    src = np.float32([[535, 460], [750, 460], [1480, 720], [-180, 720]])
    dst = np.float32([[0, 0], [1280, 0], [1280, 720], [0, 720]])
    
    perspective_warper.compute_perspective_transform(src, dst)

compute_road_perspective_matrix(persp_warper)
    
def read_perspective_warper_road_images(perspective_warper, filenames):
    undistorter = perspective_warper.undistorter
    
    result = []
    for fn in filenames:
        img = mpimg.imread(fn)
        undist = undistorter.undistort_image(img)
        warped = perspective_warper.warp_perspective(undist)
        
        if warped is None:
            continue
        
        result.append( (img, warped,) )
        
    return result

_show_dist_undist_images_by_2( read_perspective_warper_road_images(persp_warper, _build_full_paths('test_images/')), title1='Warped Image' )

In [ ]:
img = mpimg.imread('test_images/straight_lines1.jpg')
undist = undistorter.undistort_image(img)

def find_perspective_matrix(xt1,xt2,yt,xl1,xl2,yl):
    # src = np.float32([[535, 460], [750, 460], [1480, 720], [-180, 720]])
    src = np.float32([[xt1, yt], [xt2, yt], [xl2, yl], [xl1, yl]])
    dst = np.float32([[0, 0], [1280, 0], [1280, 720], [0, 720]])
    persp_warper.compute_perspective_transform(src, dst)
    
    warped = persp_warper.warp_perspective(undist)
    
    _show_dist_undist_images_by_2([(img, warped)])
    plt.show()
    
interact(find_perspective_matrix, 
            xt1=widgets.IntSlider(min=0,max=650,step=1,value=535),
            xt2=widgets.IntSlider(min=630,max=1280,step=1,value=750),
            yt=widgets.IntSlider(min=360,max=720,step=1,value=460),
            xl1=widgets.IntSlider(min=-300,max=650,step=1,value=-180),
            xl2=widgets.IntSlider(min=600,max=1550,step=1,value=1480),
            yl=widgets.IntSlider(min=360,max=800,step=1,value=720),
        )

## 3. Use gradients and color transforms to create a thresholded binary image.

In [ ]:
class ImageSobelThresholder:
    def __init__(self, sobel_kernel=9, abs_thresh=(20,100), mag_thresh=(30, 100), dir_thresh=(0.7, 1.3)):
        self.sobel_kernel = sobel_kernel
        
        # Absolute sobelthreshold
        self.abs_thresh = abs_thresh
        self.mag_thresh = mag_thresh
        self.dir_thresh = dir_thresh
        
    def abs_sobel_threshold(self, gray, orient='x'):
        # Take the derivative in x or y given orient = 'x' or 'y'
        if orient == 'x':
            sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=self.sobel_kernel)
        elif orient == 'y':
            sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=self.sobel_kernel)
    
        # Take the absolute value of the derivative or gradient
        abs_sobel = np.absolute(sobel)
    
        # Scale to 8-bit (0 - 255) then convert to type = np.uint8
        scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    
        # Create a mask of 1's where the scaled gradient magnitude is > thresh_min and < thresh_max
        binary_output = np.zeros_like(scaled_sobel)
        binary_output[(scaled_sobel >= self.abs_thresh[0]) & (scaled_sobel <= self.abs_thresh[1])] = 1

        # Return this mask as your binary_output image
        return binary_output
    
    
    def mag_sobel_threshold(self, gray):
        # Take both Sobel x and y gradients
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=self.sobel_kernel)
        sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=self.sobel_kernel)
        # Calculate the gradient magnitude
        gradmag = np.sqrt(sobelx**2 + sobely**2)
        # Rescale to 8 bit
        scale_factor = np.max(gradmag)/255. 
        gradmag = (gradmag/scale_factor).astype(np.uint8) 
        # Create a binary image of ones where threshold is met, zeros otherwise
        binary_output = np.zeros_like(gradmag)
        binary_output[(gradmag >= self. mag_thresh[0]) & (gradmag <= self.mag_thresh[1])] = 1

        # Return the binary image
        return binary_output
    
    
    def dir_sobel_threshold(self, gray):
        # Calculate the x and y gradients
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=self.sobel_kernel)
        sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=self.sobel_kernel)
        # Take the absolute value of the gradient direction, 
        # apply a threshold, and create a binary image result
        absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
        binary_output =  np.zeros_like(absgraddir)
        binary_output[(absgraddir >= self.dir_thresh[0]) & (absgraddir <= self.dir_thresh[1])] = 1

        # Return the binary image
        return binary_output
    
    
    def combined_sobel_thresholds(self, gray):
        # Apply each of the thresholding functions
        gradx = self.abs_sobel_threshold(gray, orient='x')
        grady = self.abs_sobel_threshold(gray, orient='y')
        mag_binary = self.mag_sobel_threshold(gray)
        dir_binary = self.dir_sobel_threshold(gray)
    
        combined = np.zeros_like(dir_binary)
        combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
        return combined
    
    def gray_combined_sobel_thresholds(self, undist):
        gray = cv2.cvtColor(undist, cv2.COLOR_RGB2GRAY)
        return self.combined_sobel_thresholds(gray)

In [ ]:
def apply_thresholds(undist, threshold_fct, cvt_2_gray=True):
    if cvt_2_gray:
        gray = cv2.cvtColor(undist, cv2.COLOR_RGB2GRAY)
        return threshold_fct(gray)
    else:
        return threshold_fct(undist)

def show_thresholded_binary_images(undistorter, threshold_fct, cvt_2_gray=True):
    result = []
    for fn in _build_full_paths("test_images/"):
        img = mpimg.imread(fn)
        undist = undistorter.undistort_image(img)
        
        undist = persp_warper.warp_perspective(undist)
        
        thres_img = apply_thresholds(undist, threshold_fct, cvt_2_gray)
        result.append( (img, thres_img,) )
        
    return result

sobel_thresholder = ImageSobelThresholder()

### a. Apply Gradients

#### Sobel

In [ ]:
_show_dist_undist_images_by_2( show_thresholded_binary_images(undistorter, sobel_thresholder.abs_sobel_threshold), title1='Sobel Binary Image' )

#### Gradient Magnitude

In [ ]:
_show_dist_undist_images_by_2( show_thresholded_binary_images(undistorter, sobel_thresholder.mag_sobel_threshold), title1='Gradient Magnitude Image' )


#### Gradient direction

In [ ]:
_show_dist_undist_images_by_2( show_thresholded_binary_images(undistorter, sobel_thresholder.dir_sobel_threshold), title1='Gradient Direction Binary Image' )

#### Combined gradient thresholds

In [ ]:
_show_dist_undist_images_by_2( show_thresholded_binary_images(undistorter, sobel_thresholder.combined_sobel_thresholds), title1='Combined Thresholded Binary Image' )


In [ ]:
img = mpimg.imread('test_images/test1.jpg')
undist = undistorter.undistort_image(img)
gray = cv2.cvtColor(undist, cv2.COLOR_RGB2GRAY)

def find_gradient_thresholds(sobel_kernel, abs_thresh_low, abs_thresh_high, mag_thresh_low, mag_thresh_high, dir_thresh_low, dir_thresh_high):
    sobel_kernel = int(sobel_kernel)
    sobel_thresholder = ImageSobelThresholder(sobel_kernel=sobel_kernel, abs_thresh=(abs_thresh_low,abs_thresh_high), mag_thresh=(mag_thresh_low, mag_thresh_high), dir_thresh=(dir_thresh_low, dir_thresh_high))
    binary = sobel_thresholder.combined_sobel_thresholds(gray)
    
    _show_dist_undist_images_by_2([(img, binary)])
    plt.show()
    
interact(find_gradient_thresholds, 
            sobel_kernel=widgets.Dropdown(options=['3', '5', '7', '9', '13', '15', '17', '21', '23', '25', '27'], value='9'),
            abs_thresh_low=widgets.IntSlider(min=0,max=255,step=1,value=25),
            abs_thresh_high=widgets.IntSlider(min=0,max=255,step=1,value=100),
            mag_thresh_low=widgets.IntSlider(min=0,max=255,step=1,value=30),
            mag_thresh_high=widgets.IntSlider(min=0,max=255,step=1,value=100),
            dir_thresh_low=widgets.FloatSlider(min=0,max=3.2,step=0.05,value=1.6),
            dir_thresh_high=widgets.FloatSlider(min=0,max=3.2,step=0.05,value=3.2),
        )

### b. Apply Color Transforms

#### Color Threshold

In [ ]:
class ImageColorThresholder:
    def __init__(self, ch_thresh=( ('H', (15, 100)), ('S', (90, 255)) ) ):
        self.ch_thresh = ch_thresh
        
    @staticmethod
    def color_threshold(img, channel='S', thresh=(90, 255)):
        ch_map = {
            'H': 0,
            'L': 1,
            'S': 2
        }
        hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        channel = hls[:,:,ch_map[channel]]
        
        binary = np.zeros_like(channel)
        binary[(channel > thresh[0]) & (channel <= thresh[1])] = 1
        return binary
    
    def combined_color_thresholds(self, img):
        prev_binary = None
        for ch, thresh in self.ch_thresh:
            binary = self.color_threshold(img, ch, thresh)
            if prev_binary is not None:
                binary = binary | prev_binary
            
            prev_binary = binary
    
        img_size = (img.shape[1], img.shape[0])
        result = np.zeros_like(img_size)
        result[binary] = 1
        return binary
    
color_thresholder = ImageColorThresholder()

In [ ]:
_show_dist_undist_images_by_2( show_thresholded_binary_images(undistorter, ImageColorThresholder.color_threshold, cvt_2_gray=False), title1='Color Transform' )


In [ ]:
def test_L_color_thresholding(img):
    return ImageColorThresholder.color_threshold(img, channel='S', thresh=(90, 255))

_show_dist_undist_images_by_2( show_thresholded_binary_images(undistorter, test_L_color_thresholding, cvt_2_gray=False), title1='Color Transform' )


#### Combined Color Threshold

In [ ]:
_show_dist_undist_images_by_2( show_thresholded_binary_images(undistorter, color_thresholder.combined_color_thresholds, cvt_2_gray=False), title1='Combined Color Transform' )


In [ ]:
img = mpimg.imread('test_images/test5.jpg')
undist = undistorter.undistort_image(img)

def find_color_thresholds(H_thresh, S_thresh):
    color_thresholder = ImageColorThresholder(ch_thresh = (('H', H_thresh,), ('S', S_thresh,),) )
    binary = color_thresholder.combined_color_thresholds(undist)
    
    _show_dist_undist_images_by_2([(img, binary)])
    plt.show()
    
interact(find_color_thresholds,
            H_thresh=widgets.FloatRangeSlider(value=[15, 100], min=0, max=180, step=1,),
            # L_thresh=widgets.FloatRangeSlider(value=[2, 20], min=0, max=255, step=1,),
            S_thresh=widgets.FloatRangeSlider(value=[90, 255], min=0, max=255, step=1,),
        )

## 4. 

In [ ]:
def image_processing_pipeline(undistorter, perspective_warper, sobel_thresholder, color_thresholder, img):
    undist = undistorter.undistort_image(img)
    warp_persp = perspective_warper.warp_perspective(undist)
    
    sobel_binary = sobel_thresholder.gray_combined_sobel_thresholds(warp_persp)
    color_binary = color_thresholder.color_threshold(warp_persp)
    
    binary = np.zeros_like(color_binary)
    binary[(sobel_binary == 1) | (color_binary == 1)] = 1
    
    return binary

In [ ]:
class LanesFinder:
    def __init__(self):
        pass
    
    def find_lanes(self, binary_warped, nb_window=15, margin=100, minpix=50):
        out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
        left_lane_inds, right_lane_inds = [], []
        
        # Set height of windows
        window_height = np.int(binary_warped.shape[0]/nb_window)
        # Identify the x and y positions of all nonzero pixels in the image
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        
        leftx_current, rightx_current = None, None
        
        for idx, window in enumerate(range(nb_window)):
            win_from = np.int((idx+1) * window_height)
            histogram = np.sum(binary_warped[win_from:,:], axis=0)
            
            midpoint = np.int(histogram.shape[0]/2)
            if leftx_current is None:
                leftx_current = np.argmax(histogram[:midpoint])
                
            if rightx_current is None:
                rightx_current = np.argmax(histogram[midpoint:]) + midpoint
            
            # Identify window boundaries in x and y (and right and left)
            win_y_low = binary_warped.shape[0] - (window+1)*window_height
            win_y_high = binary_warped.shape[0] - window*window_height
            win_xleft_low = leftx_current - margin
            win_xleft_high = leftx_current + margin
            win_xright_low = rightx_current - margin
            win_xright_high = rightx_current + margin
            # Draw the windows on the visualization image
            cv2.rectangle(out_img,(win_xleft_low, win_y_low),(win_xleft_high, win_y_high),(0,255,0), 2) 
            cv2.rectangle(out_img,(win_xright_low, win_y_low),(win_xright_high, win_y_high),(0,255,0), 2) 
            # Identify the nonzero pixels in x and y within the window
            good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
            good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
            # Append these indices to the lists
            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)
            # If you found > minpix pixels, recenter next window on their mean position
            if len(good_left_inds) > minpix:
                leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
            if len(good_right_inds) > minpix:        
                rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
        
        # Concatenate the arrays of indices
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)

        # Extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds] 

        # Fit a second order polynomial to each
        if lefty.size == 0 or righty.size == 0:
            return
    
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)
    
        # Visualize the draw
        # Generate x and y values for plotting
        ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

        out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    
        # return out_img, left_fitx, right_fitx
    
        plt.imshow(binary_warped)
        plt.figure()
    
        plt.imshow(out_img)
        plt.plot(left_fitx, ploty, color='yellow')
        plt.plot(right_fitx, ploty, color='yellow')
        plt.xlim(0, 1280)
        plt.ylim(720, 0)
        plt.figure()

In [ ]:
def find_lanes(binary_warped):
    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[int(binary_warped.shape[0]/2):,:], axis=0)
    
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
    # Choose the number of sliding windows
    nwindows = 18
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low, win_y_low),(win_xleft_high, win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low, win_y_low),(win_xright_high, win_y_high),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    if lefty.size == 0 or righty.size == 0:
        return
    
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Visualize the draw
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    
    # return out_img, left_fitx, right_fitx
    
    plt.imshow(binary_warped)
    plt.figure()
    
    plt.imshow(out_img)
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    plt.xlim(0, 1280)
    plt.ylim(720, 0)
    plt.figure()

In [ ]:
def find_lanes1(warped):
    window_width = 50 
    window_height = 80 # Break image into 9 vertical layers since image height is 720
    margin = 50 # How much to slide left and right for searching

    def window_mask(width, height, img_ref, center,level):
        output = np.zeros_like(img_ref)
        output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height),max(0,int(center-width/2)):min(int(center+width/2),img_ref.shape[1])] = 1
        return output

    def find_window_centroids(image, window_width, window_height, margin):
    
        window_centroids = [] # Store the (left,right) window centroid positions per level
        window = np.ones(window_width) # Create our window template that we will use for convolutions
    
        # First find the two starting positions for the left and right lane by using np.sum to get the vertical image slice
        # and then np.convolve the vertical image slice with the window template 
    
        # Sum quarter bottom of image to get slice, could use a different ratio
        l_sum = np.sum(warped[int(3*warped.shape[0]/4):,:int(warped.shape[1]/2)], axis=0)
        l_center = np.argmax(np.convolve(window,l_sum))-window_width/2
        r_sum = np.sum(warped[int(3*warped.shape[0]/4):,int(warped.shape[1]/2):], axis=0)
        r_center = np.argmax(np.convolve(window,r_sum))-window_width/2+int(warped.shape[1]/2)
    
        # Add what we found for the first layer
        window_centroids.append((l_center,r_center))
    
        # Go through each layer looking for max pixel locations
        for level in range(1,(int)(warped.shape[0]/window_height)):
            # convolve the window into the vertical slice of the image
            image_layer = np.sum(warped[int(warped.shape[0]-(level+1)*window_height):int(warped.shape[0]-level*window_height),:], axis=0)
            conv_signal = np.convolve(window, image_layer)
            # Find the best left centroid by using past left center as a reference
            # Use window_width/2 as offset because convolution signal reference is at right side of window, not center of window
            offset = window_width/2
            l_min_index = int(max(l_center+offset-margin,0))
            l_max_index = int(min(l_center+offset+margin,warped.shape[1]))
            l_center = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset
            # Find the best right centroid by using past right center as a reference
            r_min_index = int(max(r_center+offset-margin,0))
            r_max_index = int(min(r_center+offset+margin,warped.shape[1]))
            r_center = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset
            # Add what we found for that layer
            window_centroids.append((l_center,r_center))

        return window_centroids

    window_centroids = find_window_centroids(warped, window_width, window_height, margin)

    # If we found any window centers
    if len(window_centroids) > 0:

        # Points used to draw all the left and right windows
        l_points = np.zeros_like(warped)
        r_points = np.zeros_like(warped)

        # Go through each level and draw the windows 	
        for level in range(0,len(window_centroids)):
            # Window_mask is a function to draw window areas
            l_mask = window_mask(window_width,window_height,warped,window_centroids[level][0],level)
            r_mask = window_mask(window_width,window_height,warped,window_centroids[level][1],level)
            # Add graphic points from window mask here to total pixels found 
            l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
            r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255

        # Draw the results
        template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
        zero_channel = np.zeros_like(template) # create a zero color channel
        template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
        warpage = np.array(cv2.merge((warped,warped,warped)),np.uint8) # making the original road pixels 3 color channels
        output = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the orignal road image with window results
 
    # If no window centers found, just display orginal road image
    else:
        output = np.array(cv2.merge((warped,warped,warped)),np.uint8)
        
    plt.imshow(output)
    plt.title('window fitting results')
    plt.show()    
    
    return output

In [ ]:
lanes_finder = LanesFinder()

for fn in _build_full_paths('test_images/'):
    img = mpimg.imread(fn)
    
    plt.imshow(img)
    plt.figure()
    
    binary = image_processing_pipeline(undistorter, persp_warper, sobel_thresholder, color_thresholder, img)
    
    if binary is None:
        continue
    
    # find_lanes(binary)
    lanes_finder.find_lanes(binary)

In [ ]:
def apply_combined_thresholds(img):
    undist = undistort_image(mtx, dist, img)
    gray = cv2.cvtColor(undist, cv2.COLOR_RGB2GRAY)
    return combined_thresholds(gray)

for fn in _build_full_paths("test_images/"):
    img = mpimg.imread(fn)
    thres_img = apply_combined_thresholds(img)
    plt.imshow(thres_img, cmap='gray')
    plt.figure()
 